In [3]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import sys, os, math
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config,T5Model
from torch.utils.data import Dataset, DataLoader

sys.path.insert(0, '../dlp')
from data_access import PQDataAccess
from data_process import *

pd.set_option('future.no_silent_downcasting', True)
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(device)

batch_size = 64
da = PQDataAccess("/home/aac/Alireza/datasets/taxseq/corpus_1000", batch_size)
epochs= 10_000
val_epoch = 100
num_val = 25

model_name = "T5"
checkpoint_dir = f"../checkpoints/{model_name}_checkpoints"

if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)
print(checkpoint_dir)

cuda:0
../checkpoints/T5_checkpoints


In [11]:
model = T5Model(T5Config()).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
print(model.config)
print(model)
print("model:", sum(p.numel() for p in model.parameters()) / 1e6, 'M parameters')

T5Config {
  "_attn_implementation_autoset": true,
  "classifier_dropout": 0.0,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "transformers_version": "4.46.0",
  "use_cache": true,
  "vocab_size": 32128
}

T5Model(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_featu

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

def train_step(model, optimizer, da, device):
    # Zero the gradients
    optimizer.zero_grad()
    
    # Get batch and convert to tensor
    tensor_batch = T5_data_to_tensor_batch(da.get_batch())
    tensor_batch.gpu(device)
    labels = tensor_batch.taxes
    
    outputs = model(tensor_batch.seq_ids)
    print(outputs.shape)
    print(labels.shape)
    
    loss = outputs.loss
        
    # Backward pass and optimization step
    loss.backward()
    optimizer.step()

    return loss.item()

In [9]:
def generate():
    # Set the model to evaluation mode
    model.eval()
    
    test_protein_sequence = "MKTAYIAKQRQISFVKSHFSRQDIL"
    input_tensor = torch.LongTensor([encode_sequence(test_protein_sequence)]).to(device)
    
    # Generate output using the model
    with torch.no_grad():
        output_sequences = model.generate(
            input_ids=input_tensor,
            max_length=200,  # Adjust based on expected taxonomy length
        )
    
    # Decode the generated output to a string
    print(t5_tokenizer.decode(output_sequences[0], skip_special_tokens=True))
    model.train()

In [10]:
model.train()

train_losses = []

for epoch in range(epochs):
    train_loss = train_step(model, optimizer, da, device)
    train_losses.append(train_loss)
    
    if (epoch + 1) % val_epoch == 0:
        mean_train_loss = sum(train_losses[-val_epoch:]) / val_epoch
        print(f"Epoch {epoch+1}, Train Loss: {mean_train_loss:.4f}")
        generate()

OutOfMemoryError: HIP out of memory. Tried to allocate 1.91 GiB. GPU 

In [ ]:
generate()